<a href="https://colab.research.google.com/github/daniel-hain/paper_skill_development_NO/blob/main/10_embeddings_sbert_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies needed for this notebook

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!pip install -qU transformers sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer, util

In [4]:
import pickle
import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq

In [5]:
embeddings_dir = "/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/embeddings_folder"

---

## Using sentence-transformers

In [6]:
model = SentenceTransformer('NbAiLab/nb-sbert-base')

# Embedding jobs

In [7]:
# Read the CSV file
#df_jobs  = pd.read_csv("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_job_plain.csv")
prefix_save = 'update_202308'
df_jobs  = pd.read_parquet(f"/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_job_plain_{prefix_save}.parquet")

In [8]:
df_jobs.head(5)

,job_id,year,text
0,0100-2016-02-1,2016,Ønskede kvalifikasjoner ¿ Kompetanse og/eller ...
1,0100-2016-02-1,2016,¿ Kontaktskapende og inviterende holdning ¿ Ti...
2,0100-2016-02-1,2016,Ansettelsen forutsetter fremlegging av godkjen...
3,0100-2016-02-1,2016,Det er også et ønske at søkeren har førerkort ...
4,0100-2016-02-1,2016,Ytterligere opplysninger om stillingen kan fås...


In [9]:
from more_itertools import sliced
CHUNK_SIZE = 1000000  #chunk row size
index_slices = sliced(range(len(df_jobs)), CHUNK_SIZE)

In [10]:
for index_slice in index_slices:
  chunk = df_jobs.iloc[index_slice]

  embeddings = model.encode(chunk['text'].tolist(), show_progress_bar=True, convert_to_tensor=False, convert_to_numpy=True)

  df_embeddings = pd.DataFrame(embeddings)
  del embeddings
  df_embeddings = df_embeddings.set_index(chunk['job_id'])
  df_embeddings = pa.Table.from_pandas(df_embeddings)

  pq.write_table(df_embeddings, f"{embeddings_dir}/job_embeddings_{prefix_save}_{index_slice}.parquet")
  del df_embeddings
  del chunk


  print("::::: Finished chunk ", index_slice, ':::::')

Batches:   0%|          | 0/31250 [00:00<?, ?it/s]

::::: Finished chunk  range(0, 1000000) :::::


Batches:   0%|          | 0/31250 [00:00<?, ?it/s]

::::: Finished chunk  range(1000000, 2000000) :::::


Batches:   0%|          | 0/31250 [00:00<?, ?it/s]

...

::::: Finished chunk  range(10000000, 11000000) :::::


Batches:   0%|          | 0/31250 [00:00<?, ?it/s]

::::: Finished chunk  range(11000000, 12000000) :::::


Batches:   0%|          | 0/16352 [00:00<?, ?it/s]

::::: Finished chunk  range(12000000, 12523254) :::::


In [11]:
#df_embeddings = pd.DataFrame(embeddings)
#df_embeddings = df_embeddings.set_index(chunk['job_id'])
#df_embeddings = pa.Table.from_pandas(df_embeddings)
#pq.write_table(df_embeddings, f"{embeddings_dir}/job_embeddings_{index_slice}.parquet")